Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import sqlite3
import shutil
import os

Initialize SQL

In [2]:
conn = sqlite3.connect('Budget.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS budget (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Month TEXT,
    Category TEXT,
    Expense REAL)
''')

Define Months, Category, and Category Colors

In [3]:
CATEGORY_COLORS = {
    "Income": "gold",
    "Expense": "red",
    "School": "violet",
    "Retirement": "green",
    "Utilities": "purple",
    "Entertainment": "orange",
    "Phone payment": "cyan",
    "Groceries": "pink",
    "Eating Out": "brown",
    "Gas": "grey",
    "Mortgage": "lightblue",
    "Misc": "lightgreen"
}

Months = [
    "January", "Februrary", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"
]

Category = [
    "Income", "Expense", "School", "Retirement", "Utilities", "Entertainment", "Phone payment", "Groceries", "Eating Out", "Gas","Mortgage", "Misc"
]

Category Colors

Function asking for confirmation

In [4]:
def confirmation_all():
    while True:
        #Asks for confirmation. Will only take the first letter and puts it in lowercase
        confirmation = input("Is that correct?(Y/N): ").strip().lower()[0]

        if confirmation == "y":
            return True
        elif confirmation == "n":
            return False
        else:
            print("Invalid response. Please enter Y or N")

Code to ask which month to add new transaction to

In [5]:
def choose_month():
    while True:     
        try:
            #Ask for month input
            month_index = int(input("Choose a month by number (e.g. 1 for January): "))
            if 1 <= month_index <= 12:
                #Need to subtract 1 for array
                month = Months[month_index-1]
                print("You chose the month of: \n", month)
                if confirmation_all():
                    return month
                    break
                else:
                    continue
            #If number not within normal, then will ask again
            else:
                print("Invalid number. Please try again.")
        #If not a valid input, will ask again
        except ValueError:
            print("Invalid number. Please try again.")




Code to list and ask what category to add 

In [6]:
def category_type():
    print("\nCategories:")
    for i, categories in enumerate(Category, start = 1):
        print(f"{i}. {categories}")
    while True:
        try:
            category_index = int(input("Choose a category to add to: "))
            if 1 <= category_index <= 12:
                #Need to subtract 1 for array
                category = Category[category_index - 1]
                print("You have chosen: ", category)
                if confirmation_all():
                    return category
                    break
                else:
                    continue                
            else:
                print("Invalid number. Please try again.")
        except ValueError:
            print("Invalid number. Please try again.")

Asking how much to add to the category and then repeating until finished.

In [7]:
def expense_amount(month, category):
    try:
        cursor.execute("SELECT Expense FROM budget WHERE Month = ? AND Category = ?", (month, category))
        existing_entry = cursor.fetchone()
        
        if existing_entry:
            # If the entry exists, update the expense
            existing_expense = existing_entry[0]
            new_expense = float(input("How much was the expense?: "))
            updated_expense = existing_expense + abs(new_expense)
            cursor.execute("UPDATE Budget SET Expense = ? WHERE Month = ? AND Category = ?", (updated_expense, month, category))
            print(f"Expense for {category} in {month} updated to: {updated_expense}")
            conn.commit()
            return updated_expense
        else:
            # If the entry doesn't exist, add a new one
            expense = float(input("How much was the expense?: "))
            cursor.execute("INSERT INTO Budget (Month, Category, Expense) VALUES (?, ?, ?)", (month, category, abs(expense)))
            print(f"Transaction added: {month} - {category} - {expense}")
            conn.commit()
            return abs(expense)
    except ValueError:
        print("Please enter a valid value.")



Append the transaction

In [8]:
def add_transaction():
    month = choose_month()
    category = category_type()
    expense = expense_amount(month, category)
    
    # Check if entry already exists in the database
    cursor.execute("SELECT * FROM Budget WHERE Month = ? AND Category = ?", (month, category))
    existing_entry = cursor.fetchone()

    if existing_entry:
        # If entry exists, update the existing expense
        cursor.execute("UPDATE Budget SET Expense = ? WHERE Month = ? AND Category = ?", (expense, month, category))
        print(f"Expense for {category} in {month} updated to: {expense}")
    else:
        # If no existing entry, add a new one
        cursor.execute("INSERT INTO Budget (Month, Category, Expense) VALUES (?, ?, ?)", (month, category, expense))
        print(f"Transaction added: {month} - {category} - {expense}")

    # Commit changes to the database
    conn.commit()


Create chart for income/expenses

In [9]:
def plot_pie_chart():

    cursor.execute('''
            SELECT Month, Category, SUM(Expense)
            FROM Budget
            Group by Month, Category   
    ''')
    rows = cursor.fetchall()
    # Group by category and month and sum expenses for each category
    if rows:
        df = pd.DataFrame(rows, columns = ['Month', 'Category', 'Expense'])
        expense_data = df.groupby(['Month', 'Category'])['Expense'].sum().unstack(fill_value=0)

    # Loop through each month and plot its pie chart
        for month in expense_data.index:
        # Extract expenses for the current month as a pandas Series
            monthly_expenses = expense_data.loc[month]

        # Filter out zero-value categories
            non_zero_expenses = monthly_expenses[monthly_expenses != 0]

        # Plot a pie chart for the current month
            plt.figure(figsize=(8, 8))
            wedges, texts, autotexts = plt.pie(
            non_zero_expenses, 
            labels=non_zero_expenses.index, 
            autopct='%1.1f%%', 
            startangle=140,
            colors=[CATEGORY_COLORS[category] for category in non_zero_expenses.index]
    )

        # Adjust the autotexts to show the actual value along with the percentage
            for i, autotext in enumerate(autotexts):
                value = non_zero_expenses.iloc[i]
                autotext.set_text(f"{autotext.get_text()} (${value:.2f})")

    # Customize the texts and autotexts appearance
            for text in texts:
                text.set_fontsize(12)
                text.set_color("blue")
            for autotext in autotexts:
                autotext.set_fontsize(10)
                autotext.set_color("darkgreen")

    # Show the pie chart
            plt.title(f"Monthly Expenses for {month}")
    plt.show()

Function for viewing data

In [10]:
def view_data():
    cursor.execute("SELECT * FROM Budget")
    rows = cursor.fetchall()

    if rows:
        budget_data = pd.DataFrame(rows, columns = ['ID', 'Month', 'Category', 'Expense'])
        print(budget_data)
    else:
        print("No Data Available")

Main Code

In [ ]:
def main():
    global budget_data
    if os.path.exists('/kaggle/working/Budget.db'):
        shutil.move('/kaggle/working/Budget.db', 'Budget.db')
    print("Budget Program")

    while True:
        print("\nOptions: 1. Add Transaction 2. View Data 3. Exit 4. Clear Data 5. Print Expenses")
        choice = input("Please choose an option: ")

        if choice == '1':
            add_transaction()
        elif choice == '2':
            view_data()
        elif choice == '3':
            shutil.move('Budget.db', '/kaggle/working/Budget.db')
            break
        elif choice == '4':
            cursor.execute("DELETE FROM budget")
            conn.commit()
            print("All data cleared.")
        elif choice == '5':
            plot_pie_chart()
        else:
            print("Invalid option. Please try again.")

if __name__ == "__main__":
    main()

conn.close()

Budget Program

Options: 1. Add Transaction 2. View Data 3. Exit 4. Clear Data 5. Print Expenses


Please choose an option:  2


No Data Available

Options: 1. Add Transaction 2. View Data 3. Exit 4. Clear Data 5. Print Expenses


Please choose an option:  1.


Invalid option. Please try again.

Options: 1. Add Transaction 2. View Data 3. Exit 4. Clear Data 5. Print Expenses


Please choose an option:  1
Choose a month by number (e.g. 1 for January):  1


You chose the month of: 
 January


Is that correct?(Y/N):  y



Categories:
1. Income
2. Expense
3. School
4. Retirement
5. Utilities
6. Entertainment
7. Phone payment
8. Groceries
9. Eating Out
10. Gas
11. Mortgage
12. Misc
